<a href="https://colab.research.google.com/github/SYEON9/natural_language_3th/blob/main/HW/%5BHW15%5DVisualizaing_MNIST_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dimensionality Reduction

인간은 고차원 공간을 이해할 수 없기 때문에 이해할 수 있는 저차원으로 변환하는 기술을 연구했다. 이것을 dimensionality reduction이라고 한다. 이것을 이번 실습에서 직접적으로 이해해보자. 이번 실습은 MNIST dataset을 활용하여 수행해보겠다.


In [ ]:
#데이터 불러오기
#사용할 데이터는 MNIST Dataset이다. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', cache = False)     #openml에서 데이터셋 이름으로 데이터 불러오기

In [ ]:
#독립변수와 종속변수로 나누기
#astype(): 데이터 프레임의 모든 타입이 같을 때 한번에 변환하기
x = mnist.data.astype('float32').to_numpy()
y = mnist.target.astype('int64').to_numpy()

In [ ]:
x.shape

In [ ]:
#이미지 하나를 선택하여 데이터 확인하기
plt.figure(figsize = (5,5))
idx = 5

grid_data = x[idx].reshape(28,28)      #이미지가 28x28이므로 784차원의 벡터를 가진다. 
plt.imshow(grid_data, interpolation = 'none') 
plt.show()

print('label : {}'.format(y[idx]))

In [ ]:
plt.clf()      #matplotlib의 전체 그림을 지움. 
plt.figure(figsize = (5,5))

rand_img = np.random.rand(28,28)
plt.imshow(rand_img)

plt.show()

Visualization using PCA

데이터가 가장 흩어져있는 축을 찾아 projection하여 차원의 개수를 줄이는 방법이다. 우리는 데이터의 분산이 큰 축을 찾는 것을 목표로 한다. PCA는 scikit-learn 패키지를 활용하자.

In [ ]:
#70000개의 총 데이터를 사용하기엔 데이터 개수가 너무 많으므로 15000개만 사용해보자. 
labels = y[:15000]
data=x[:15000]

print('the shape of sample data = ', data.shape)

In [ ]:
#데이터의 개수가 매우 많으므로 정규화도 시켜주자.
#z-score normalization
from sklearn.preprocessing import StandardScaler
standardized_data = StandardScaler().fit_transform(data)
print(standardized_data)

In [ ]:
sample_data  = standardized_data

In [ ]:
#이제 PCA를 적용해보자.
from sklearn import decomposition

pca = decomposition.PCA()

In [ ]:
#우리는 feature의 차원을 2차원으로 축소할 것이다. 
pca.n_components = 2
pca_data = pca.fit_transform(sample_data)   #sample data에 pca 적용.

#pca를 적용하여 만들어진 데이터를 살펴보자.
print('shape of pca_reduced.shape = ', pca_data.shape)    #데이터의 차원이 2차원으로 감소한 것을 알 수 있다.

In [ ]:
pca_data.T.shape

In [ ]:
labels.shape

In [ ]:
np.vstack((pca_data.T, labels)).shape

PCA 시각화

각 label마다 색을 부여하여 시각화해보자. 

In [ ]:
#먼저 2차원에 label을 추가한다. 
pca_data = np.vstack((pca_data.T, labels)).T     #배열을 row로 결합.

In [ ]:
import seaborn as sn

#배열을 데이터프레임으로 만들자.
pca_df = pd.DataFrame(data=pca_data, columns = ('1st_principal','2nd_principal','label'))

#시각화하기
#FacetGrid(): 다양한 범주형 값을 가지는 데이터를 시각화하는데 좋은 방법으로 행과 열 방향으로 서로 다른 조건을 적용하여 여러 개의 서브 플롯을 제작.
sn.FacetGrid(pca_df, hue = 'label',size=6).map(plt.scatter, '1st_principal','2nd_principal')   #hue: 범례 추가
plt.show()

이번에는 직접 eigen vector를 구하고 데이터를 사영시켜 차원 축소를 진행시켜보자.



In [ ]:
#1. eigen value와 eigen vector를 구하기 위해 공분산 행렬 구하자.
covar_matrix = np.matmul(sample_data.T, sample_data)
print("The shape of variance martix = ", covar_matrix.shape)

In [ ]:
sample_data.mean()

In [ ]:
# 2.공분산 행렬을 이용해 scipy 패키지 안의 eigh 함수를 통해 eigen value와 eigen vector를 구하자
#2D로 차원을 축소할 것이므로 가장 큰 두개의 값을 선정해서 구하자
from scipy.linalg import eigh

values, vectors = eigh(covar_matrix, eigvals = (782,783))
print("Shape of eigen vectors = ", vectors.shape)

vectors = vectors.T
print("Updated shape of eigen vectors = ",vectors.shape)

In [ ]:
# 3. 이제 eigen vector를 축으로 데이터를 사영시키자
import matplotlib.pyplot as plt

new_coordinates = np.matmul(vectors, sample_data.T)
print(" Resultant new data points' shape ", vectors.shape, "x", sample_data.T.shape,'=', new_coordinates.shape)

In [ ]:
labels.shape

In [ ]:
#이제 우리 데이터는 784차원에서 2차원으로 줄어들었다.
#아까처럼 시각화해보자.
import pandas as pd

new_coordinates = np.vstack((new_coordinates, labels.reshape(1,-1))).T

dataframe = pd.DataFrame(data= new_coordinates, columns = ('1st_principal','2nd_principal','label'))
print(dataframe.head())

In [ ]:
import seaborn as sn
sn.FacetGrid(dataframe, hue = 'label', size = 6).map(plt.scatter, '1st_principal','2nd_principal').add_legend()
plt.show()